In [1]:
import composuite
from diffusion.utils import *
from CORL.algorithms.offline.td3_bc import *
from CORL.shared.buffer import *
from CORL.shared.logger import *

from diffusion.utils import *
from collections import defaultdict
import composuite
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

In [ ]:
robot = 'IIWA'
obj = 'Box'
obst = 'None'
subtask = 'Trashcan'

representative_indicators_env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=True, ignore_done=False)
modality_dims = representative_indicators_env.modality_dims


base_agent_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/data'
dataset = load_single_composuite_dataset(base_path=base_agent_data_path, 
                                            dataset_type='expert', 
                                            robot=robot, obj=obj, 
                                            obst=obst, task=subtask)
agent_dataset = transitions_dataset(dataset)
agent_dataset, _ = remove_indicator_vectors(modality_dims, agent_dataset)
agent_obs = agent_dataset['observations']
agent_actions = agent_dataset['actions']
agent_next_obs = agent_dataset['next_observations']
agent_rewards = agent_dataset['rewards']
agent_terminals = agent_dataset['terminals']
agent_dataset = make_inputs(agent_dataset)

base_synthetic_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/cluster_results/diffusion/cond_diff_20/test/'
synthetic_dataset = load_single_synthetic_dataset(base_path=base_synthetic_data_path, 
                                                  robot=robot, obj=obj, 
                                                  obst=obst, task=subtask)
synthetic_obs = synthetic_dataset['observations']
synthetic_actions = synthetic_dataset['actions']
synthetic_next_obs = synthetic_dataset['next_observations']
synthetic_rewards = synthetic_dataset['rewards']
synthetic_terminals = synthetic_dataset['terminals']
synthetic_dataset = make_inputs(synthetic_dataset)

In [ ]:
agent_dataset.shape, synthetic_dataset.shape

In [ ]:
agent_actions.shape, synthetic_actions.shape

In [ ]:
env = composuite.make(robot, obj, obst, subtask, use_task_id_obs=False, ignore_done=False)
print(env.modality_dims)

In [ ]:
cumulative_ranges = {}
start_idx = 0
for key, dim in env.modality_dims.items():
    end_idx = start_idx + dim[0]
    cumulative_ranges[key] = (start_idx, end_idx)
    start_idx = end_idx

dim_names = {}
for dim, val_range in cumulative_ranges.items():
    print(dim, val_range)
    for idx in range(val_range[0], val_range[1]):
        dim_names[idx] = dim

In [ ]:
dataset1 = agent_obs
dataset2 = synthetic_obs

num_dimensions = dataset1.shape[1]

for idx in range(num_dimensions):
    plt.figure(figsize=(8, 6))
    plt.hist(dataset1[:, idx], bins=50, alpha=0.5, label='Agent', color='blue', density=True)
    plt.hist(dataset2[:, idx], bins=50, alpha=0.5, label='Synthetic', color='orange', density=True)
    plt.title(f"{dim_names[idx]}, {idx+1}")
    plt.legend()
    plt.show()

In [ ]:
state_mean, state_std = compute_mean_std(agent_dataset, eps=1e-3)
print(state_mean.mean(), state_std.mean())

In [ ]:
state_mean, state_std = compute_mean_std(synthetic_dataset, eps=1e-3)
print(state_mean.mean(), state_std.mean())

In [10]:
random_indices = np.random.choice(agent_dataset.shape[0], 1000, replace=False)
sampled_agent_data = agent_dataset[random_indices]

random_indices = np.random.choice(synthetic_dataset.shape[0], 1000, replace=False)
sampled_synthetic_data = synthetic_dataset[random_indices]

In [ ]:
print(sampled_agent_data.shape, sampled_synthetic_data.shape)

In [ ]:
for idx in range(sampled_agent_data.shape[1]):
    print(idx)
    print('Mean:', sampled_agent_data[:, idx].mean(), sampled_synthetic_data[:, idx].mean())
    print('Std:', sampled_agent_data[:, idx].std(), sampled_synthetic_data[:, idx].std())

In [13]:
combined_data = np.concatenate((sampled_agent_data, sampled_synthetic_data), axis=0)

agent_origin = np.zeros(sampled_agent_data.shape[0], dtype=int)  # 0 for expert
synthetic_origin = np.ones(sampled_synthetic_data.shape[0], dtype=int)   # 1 for synthetic
combined_origins = np.concatenate((agent_origin, synthetic_origin), axis=0)

In [14]:
normalized_data = StandardScaler().fit_transform(combined_data)
tsne = TSNE(n_components=2, random_state=42)
embeddings = tsne.fit_transform(normalized_data)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(embeddings[combined_origins == 0, 0], embeddings[combined_origins == 0, 1], 
            c='blue', label='Agent', alpha=0.6, edgecolor='k', s=20)
plt.scatter(embeddings[combined_origins == 1, 0], embeddings[combined_origins == 1, 1], 
            c='red', label='Diffusion', alpha=0.6, edgecolor='k', s=20)
plt.title('t-SNE')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.legend()
plt.show()